#  🤗 Transformers Finetuning

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://huggingface.co/docs/transformers/training
* https://huggingface.co/docs/datasets/main/en/repository_structure
* https://huggingface.co/docs/datasets/main/en/package_reference/loading_methods#datasets.load_dataset
* https://huggingface.co/docs/transformers/v4.35.2/en/training#prepare-a-dataset
* https://huggingface.co/docs/datasets/process
* https://huggingface.co/docs/evaluate/index
* https://huggingface.co/docs/transformers/main_classes/trainer
* https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer#transformers.TrainingArguments

## Задачи для совместного разбора

1\. Обсудите основные шаги по дообучению моделей из экосистемы 🤗 Transformers.

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Разбейте данные из файла `reviews_polarity.csv` на обучающее и валидационное множество в соотношении 80 на 20. Создайте папку `reviews_polarity_dataset` и сохраните в нее полученные фрагменты данных под названием `train.csv` и `test.csv`. Создайте объект `datasets.Dataset`, используя функцию `load_dataset`.

Токенизируйте строки при помощи токенизатора, соотвествующего модели `rubert-base-cased-sentiment`. Удалите из датасета поле `text` после токенизации, замените поле `class` на `labels` и приведите данные к тензорам `torch`.

Создайте два `DataLoader` на основе обучающего и валидационного множества. Получите батч из обучающего множества и выведите его на экран.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch as th

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/пм21_финашка/3 курс/NLP/05_transformers/reviews_polarity.csv')
df.head()

,text,class
0,"Очень хорошо что открылась 5 ка, теперь не над...",1
1,"Тесно, вечная сутолока, между рядами трудно ра...",0
2,Магазин в пешей доступности. После ремонта и р...,1
3,Магазин хороший цены и скидки нормальные токо ...,1
4,Сложно найти в торговом центре. А магазин - норм),1


In [ ]:
train, test = train_test_split(
    df,
    test_size = 0.2,
    random_state = 42
)

In [ ]:
import os

In [ ]:
os.makedirs("/content/reviews_polarity_dataset", exist_ok=True)

In [ ]:
train.to_csv('/content/reviews_polarity_dataset/train.csv')
test.to_csv('/content/reviews_polarity_dataset/test.csv')

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


In [ ]:
tokenizer

BertTokenizerFast(name_or_path='blanchefort/rubert-base-cased-sentiment', vocab_size=100792, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv",
                       data_files={
                                  "train": "/content/reviews_polarity_dataset/train.csv",
                                   "test": "/content/reviews_polarity_dataset/test.csv"
                                   }
                       )


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'class'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'class'],
        num_rows: 7644
    })
})

In [ ]:
dataset['train']['text'][:3]

['Узкие проходы между рядами,с коляской очень не удобно.',
 'Очень маленькое помещение',
 'Стандартная пятёрочка. Место довольно много. Ассортимент неплохой']

In [ ]:
# tokenizer(dataset['train']['text'])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding= True, truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=1000000)
tokenized_datasets

Map:   0%|          | 0/30574 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7644 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7644
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['text', 'Unnamed: 0'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7644
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column('class', 'labels')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7644
    })
})

In [ ]:
tokenized_datasets['train']['labels'][:3]

[0, 1, 1]

In [ ]:
tokenized_datasets = tokenized_datasets.with_format("torch", columns=['input_ids', "labels"],)
tokenized_datasets['train']['input_ids'][0]

tensor([  101, 14220,  1297, 83699,  1862, 56278,   128,   336, 74158,  1094,
          802,  4980,   132,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)  #.select(range(1000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42)   #.select(range(1000))
train_dataset, test_dataset

(Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 30574
 }),
 Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 7644
 }))

In [ ]:
from  torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

In [ ]:
train_dataset['input_ids'][0]

tensor([  101, 31861,   803,   128,  4980,   106,   106,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
next(iter(train_loader))

{'labels': tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
         1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 1, 1, 1, 1, 0, 1, 1]),
 'input_ids': tensor([[  101, 31861,   803,  ...,     0,     0,     0],
         [  101,  4054,  7461,  ...,     0,     0,     0],
         [  101, 67432,  4883,  ...,     0,     0,     0],
         ...,
         [  101,  3065,  1979,  ...,     0,     0,     0],
         [  101,  2242, 14600,  ...,     0,     0,     0],
         [  101, 41443, 48890,  ...,     0,     0,     0]])}

<p class="task" id="2"></p>

2\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя стандартный цикл обучения `torch`, настройте модель для решения задачи бинарной классификации. Во время обучения выводите на экран значение функции потерь (используйте готовые значения, которые генерирует модель) на обучающем множестве и f1 на валидационном множестве.

Здесь и далее для ускорения процесса обучения вы можете заморозить часть сети или уменьшить размер наборов данных, выбрав небольшое подмножество примеров.

- [ ] Проверено на семинаре

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2  # бинарная классификация
# model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, num_labels = num_labels, ignore_mismatched_sizes=True)

In [ ]:

import torch.nn as nn
class Neuron(nn.Module):

  def __init__(self, num_labels):
    super().__init__()
    self.model = model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, num_labels = num_labels, ignore_mismatched_sizes=True)
    for param in self.model.bert.parameters():
        param.requires_grad = False    # Заморозка

    self.loss = nn.CrossEntropyLoss()

  def forward(self, inputs, labels = None):
    out = self.model(inputs)
    pred = out.logits
    loss = self.loss(pred, labels)

    return  pred, loss

In [ ]:
model = Neuron(num_labels = num_labels).cuda()
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at blanchefort/rubert-base-cased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Neuron(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=T

In [ ]:
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import f1_score


optimizer = optim.Adam(model.parameters(), lr = 0.01)
n_epochs = 5
model = model.cuda()


for epoch in range(n_epochs):

  for i in train_loader:

    out, loss = model(i['input_ids'].cuda(), i['labels'].cuda())
  #   # print(pred, loss)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad

  for i in test_loader:

    out, loss = model(i['input_ids'].cuda(), i['labels'].cuda())
    pred = th.argmax(out, dim = 1)
    # print(pred.cpu())
    # print(i['labels'].cpu())
    f1 = f1_score(i['labels'].cpu(), pred.cpu())

  print(f'Epoch №{epoch} Loss --> {loss.item()}, f1_score for test --> {f1}')


Epoch №0 Loss --> 102.3443832397461, f1_score for test --> 0.8780487804878049
Epoch №1 Loss --> 63.798431396484375, f1_score for test --> 0.8780487804878049
Epoch №2 Loss --> 266.16888427734375, f1_score for test --> 0.8848484848484849
Epoch №3 Loss --> 173.6772918701172, f1_score for test --> 0.8780487804878049
Epoch №4 Loss --> 76.6983413696289, f1_score for test --> 0.8780487804878049


<p class="task" id="3"></p>

3\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя `transformers.Trainer`, настройте модель для решения задачи бинарной классификации. При настройке `Trainer` укажите количество эпох, равное 5. Во время обучения выводите на экран значение функции потерь на обучающем множестве и f1 на валидационном множестве.  

- [ ] Проверено на семинаре


In [ ]:
pip install accelerate -U

In [ ]:
import accelerate
accelerate.__version__

'0.25.0'

In [ ]:
pip install transformers[torch]

In [ ]:
!pip install accelerate

In [ ]:
!pip install --upgrade accelerate

In [ ]:
!pip install evaluate

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="result_trainer",
                                  num_train_epochs=5,
                                  evaluation_strategy="epoch"
                                  )

In [ ]:
import evaluate
metric = evaluate.load("f1")


def compute_f1(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  return {"f1": f1_score(labels, preds, average='weighted')}




In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
train_dataset, test_dataset

(Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1000
 }),
 Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1000
 }))

In [ ]:
trainer = Trainer(
    model=model.cuda(),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_f1
    )



In [ ]:
from sklearn.metrics import f1_score

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.547625,0.667222
2,No log,0.543120,0.667222
3,No log,0.541798,0.667222
4,0.523400,0.541815,0.667222
5,0.523400,0.541706,0.667222


TrainOutput(global_step=625, training_loss=0.5276687866210937, metrics={'train_runtime': 114.1845, 'train_samples_per_second': 43.789, 'train_steps_per_second': 5.474, 'total_flos': 151598551230000.0, 'train_loss': 0.5276687866210937, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5417056679725647,
 'eval_f1': 0.6672217194570136,
 'eval_runtime': 2.8579,
 'eval_samples_per_second': 349.902,
 'eval_steps_per_second': 43.738,
 'epoch': 5.0}

<p class="task" id="4"></p>

4\. Используя эмбеддинги `distiluse-base-multilingual-cased-v1` из пакета `sentence_transformers`, решите задачу бинарной классификации. Для этого добавьте несколько полносвязных слоев поверх модели `SentenceTransformer`. Заморозьте часть модели, отвечающей за генерацию эмбеддингов. Во время обучения выводите на экран значение функции потерь на обучающем множестве и f1 на валидационном множестве.  

- [ ] Проверено на семинаре

In [ ]:
from sentence_transformers import SentenceTransformer
class Net(nn.Module):
  def __init__(self):
    self.base_model = SentenceTransformer('..')
    self.classifier = nn.Sequential(
        nn.Linear(?, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    )

## Обратная связь
- [ ] Хочу получить обратную связь по решению